In [2]:
import pyodbc
server = 'DIST-6-505.uopnet.plymouth.ac.uk'
database = 'COMP2001_MFerguson'
username = 'MFerguson'
password = 'GjiF140*'
driver = '{ODBC Driver 17 for SQL Server}'

conn_str = (
    f'DRIVER={driver};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'UID={username};'
    f'PWD={password};'
    'Encrypt=Yes;'
    'TrustServerCertificate=Yes;'
    'Connection Timeout=30;'
    'Trusted_Connection=No;'
)
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

In [4]:
cursor.execute('''
CREATE TABLE CW2.[User] (
    user_id VARCHAR(8) PRIMARY KEY,
    username VARCHAR(255) NOT NULL,
    email VARCHAR(255) NOT NULL,
    [password] VARCHAR(255) NOT NULL,
    role VARCHAR(8) NOT NULL,
    
    CHECK (user_id LIKE '[U][S][R][0-9][0-9][0-9][0-9][0-9]'),
    CHECK (LEN(username) >= 5 AND LEN(username) <= 255),
    CHECK (email LIKE '%_@_%.__%'),
    CHECK (LEN([password])>=8),
    CHECK (role = 'admin' OR role = 'user')
)
''')
conn.commit()

In [5]:
cursor.execute('''
INSERT INTO CW2.[User] (user_id, username, email, [password], role)
VALUES
('USR00001','Admin','admin@gmail.com','AdM1NpQsW0Rd','admin'),
('USR00002','Tim Berners-Lee','tim@plymouth.ac.uk','COMP2001!', 'user'),
('USR00003','Grace Hopper','grace@plymouth.ac.uk','ISAD123!', 'user'),
('USR00004','Ada Lovelace','ada@plymouth.ac.uk','insecurePassword','user');
''')
conn.commit()

In [6]:
cursor.execute('''
CREATE TABLE CW2.Location_Point (
    location_point_id VARCHAR(8) PRIMARY KEY,
    latitude DECIMAL(9,6) NOT NULL,
    longitude DECIMAL(9,6) NOT NULL,
    description VARCHAR(255),      
    CHECK (location_point_id LIKE '[L][P][T][0-9][0-9][0-9][0-9][0-9]'),
    CHECK (latitude BETWEEN -90 AND 90),
    CHECK (longitude BETWEEN -180 AND 180),             
)
''')
conn.commit()

In [7]:
cursor.execute('''
INSERT INTO CW2.Location_Point (location_point_id, latitude, longitude, description)
VALUES
('LPT00001', 50.123456, -4.234567, 'Scenic viewpoint on the hill'),
('LPT00002', 51.654321, -3.876543, 'Lake surrounded by forest'),
('LPT00003', 52.987654, -2.123456, 'Historic site with ruins'),
('LPT00004', 53.345678, -1.654321, 'Trailhead with parking area');
''')
conn.commit()


In [11]:
cursor.execute('''
CREATE TABLE CW2.Trail (
    trail_id VARCHAR(8) PRIMARY KEY, 
    trail_name VARCHAR(255) NOT NULL UNIQUE,
    summary VARCHAR(255),
    description VARCHAR(1500),
    location VARCHAR(255) NOT NULL,
    traffic VARCHAR(8) NOT NULL,
    difficulty VARCHAR(8) NOT NULL,
    length DECIMAL(7,2) NOT NULL,           
    duration VARCHAR(5) NOT NULL,
    elevation_gain INT NOT NULL,
    route_type VARCHAR(14) NOT NULL,
               
    owner_id VARCHAR(8) NOT NULL, 
    location_pt_1 VARCHAR(8) NOT NULL,
    location_pt_2 VARCHAR(8) NOT NULL,
    location_pt_3 VARCHAR(8) NOT NULL,
    location_pt_4 VARCHAR(8) NOT NULL,
    location_pt_5 VARCHAR(8) NOT NULL,
                   
    CHECK (trail_id LIKE '[T][R][L][0-9][0-9][0-9][0-9][0-9]'), 
    CHECK (LEN(trail_name) >= 5 AND LEN(trail_name) <= 50),
    CHECK (LEN(summary) >= 10),
    CHECK (LEN(description) >= 50 OR LEN(description) = 0),
    CHECK (lower(traffic)= 'heavy' OR lower(traffic) = 'moderate' OR lower(traffic) = 'light'),
    CHECK (lower(difficulty) = 'hard' OR lower(difficulty) = 'moderate' OR lower(difficulty) = 'easy'),  
    CHECK (length > 0 AND length = ROUND(length, 2)),
    CHECK (duration LIKE'[0-9][0-9]:[0-9][0-9]'),
    CHECK (elevation_gain >=0),
    CHECK (lower(route_type) = 'out & back' OR lower(route_type) = 'loop' OR
    lower(route_type) = 'point to point'),
               
    CONSTRAINT FK_owner_id FOREIGN KEY (owner_id) REFERENCES CW2.[User](user_id),
    CONSTRAINT fk_location_pt_1 FOREIGN KEY (location_pt_1) REFERENCES CW2.Location_Point(location_point_id),
    CONSTRAINT fk_location_pt_2 FOREIGN KEY (location_pt_2) REFERENCES CW2.Location_Point(location_point_id),
    CONSTRAINT fk_location_pt_3 FOREIGN KEY (location_pt_3) REFERENCES CW2.Location_Point(location_point_id),
    CONSTRAINT fk_location_pt_4 FOREIGN KEY (location_pt_4) REFERENCES CW2.Location_Point(location_point_id),
    CONSTRAINT fk_location_pt_5 FOREIGN KEY (location_pt_5) REFERENCES CW2.Location_Point(location_point_id)

)
''')
conn.commit()


ProgrammingError: ('42S01', "[42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is already an object named 'Trail' in the database. (2714) (SQLExecDirectW)")

In [20]:
cursor.execute('''
INSERT INTO CW2.Trail (
    trail_id, trail_name, summary, description, location, traffic,
    difficulty, length, duration, elevation_gain, route_type, owner_id,
    location_pt_1, location_pt_2, location_pt_3, location_pt_4, location_pt_5
)
VALUES
('TRL00001', 'Forest Walk', 'A scenic walk through dense forest',
 'This trail takes you through a beautiful forest with diverse wildlife and flora.', 'Plymouth, Devon, UK','moderate',
 'easy', 5.50, '02:30', 300, 'loop', 'USR00002',
 'LPT00001', 'LPT00002', 'LPT00003', 'LPT00004', 'LPT00001'),
('TRL00002','Mountain Hike', 'A challenging hike with rewarding views',
 'A strenuous but rewarding trail up the mountain with breathtaking views at the summit.', 'Los Angeles, California, USA', 'heavy', 
 'hard', 12.30, '05:45', 1200, 'out & back', 'USR00003',
 'LPT00002', 'LPT00004', 'LPT00001', 'LPT00002', 'LPT00003'),
('TRL00003', 'Coastal Path', 'A leisurely walk along the coast', 
 'This trail offers stunning views of the ocean and a chance to visit sandy beaches.', 'Lisbon, Portugal', 'light',
 'moderate', 8.70, '03:15', 150, 'point to point', 'USR00004',
 'LPT00003', 'LPT00004', 'LPT00001', 'LPT00002', 'LPT00003'),
('TRL00004', 'River Valley Trek', 'A trek along a scenic river valley',
 'Follow the river through a picturesque valley with plenty of photo opportunities.', 'Moscow, Russia','moderate',
 'easy', 10.20, '04:00', 400, 'loop', 'USR00002',
 'LPT00004', 'LPT00001', 'LPT00002', 'LPT00003', 'LPT00004');

''')
conn.commit()

In [21]:
cursor.execute('''
CREATE TABLE CW2.Attraction (
    attraction_id VARCHAR(8) PRIMARY KEY,
    attraction_name VARCHAR(50) NOT NULL,

    CHECK (attraction_id LIKE '[A][T][T][0-9][0-9][0-9][0-9][0-9]'),
    CHECK (LEN(attraction_name) >= 3)               
)
''')
conn.commit()

In [22]:
cursor.execute('''
INSERT INTO CW2.Attraction (attraction_id, attraction_name)
VALUES
('ATT00001', 'Waterfall'),
('ATT00002', 'Viewpoint'),
('ATT00003', 'Historic Ruins'),
('ATT00004', 'Picnic Spot');
''')
conn.commit()


In [23]:
cursor.execute('''
CREATE TABLE CW2.Trail_Attraction (
    trail_id VARCHAR(8) NOT NULL,
    attraction_id VARCHAR(8) NOT NULL,
    PRIMARY KEY (trail_id, attraction_id),
    
    CONSTRAINT FK_trail_id_attraction FOREIGN KEY (trail_id) REFERENCES CW2.Trail(trail_id),
    CONSTRAINT FK_attraction_id FOREIGN KEY (attraction_id) REFERENCES CW2.Attraction(attraction_id)
)
''')
conn.commit()

In [24]:
cursor.execute('''
INSERT INTO CW2.Trail_Attraction (trail_id, attraction_id)
VALUES
('TRL00001', 'ATT00001'),
('TRL00002', 'ATT00002'),
('TRL00003', 'ATT00003'),
('TRL00004', 'ATT00004'),
('TRL00001', 'ATT00002'),
('TRL00002', 'ATT00004'),
('TRL00003', 'ATT00001');
''')
conn.commit()



In [3]:
cursor.execute('''
DROP TABLE CW2.Trail_Attraction
DROP TABLE CW2.Trail
DROP TABLE CW2.Location_Point
DROP TABLE CW2.Attraction
DROP TABLE CW2.Location
DROP TABLE CW2.[User]
''')
conn.commit()